# 1 Import Packages

In [31]:
import xarray as xr
import numpy as np
import os
import datetime


import src.config as config
import src.utils as utils

import math
from tqdm import tqdm

import tensorflow as tf

# 2 Metadata

In [32]:
lev_index   = 0
kfold_index = 0
experiment_name = "cv_samplestandardized"
datetime_string = datetime.datetime.now().strftime("%d_%m_%Y_%H_%M_%S")


In [33]:
model_name = "conv_level_{}_kfold_{}_date_{}".format(lev_index, kfold_index, datetime_string)
model_path = os.path.join(config.model_path, experiment_name, datetime_string,  model_name)
os.makedirs(model_path, exist_ok=True)

In [34]:
ml_transform_path = os.path.join(config.data_pro_path,"ml_transform", experiment_name)

In [35]:
train_x_filename = "train_x_lev_{}_{}.nc".format(lev_index, kfold_index)
valid_x_filename = "valid_x_lev_{}_{}.nc".format(lev_index, kfold_index)

train_y_filename = "train_y_{}_{}.nc".format(lev_index, kfold_index)
valid_y_filename = "valid_y_{}_{}.nc".format(lev_index, kfold_index)

# 3 Load Data

In [36]:
train_x_xr = xr.open_dataset(os.path.join(ml_transform_path, train_x_filename))
valid_x_xr = xr.open_dataset(os.path.join(ml_transform_path, valid_x_filename))

In [37]:
train_y_xr = xr.open_dataset(os.path.join(ml_transform_path, train_y_filename))
valid_y_xr = xr.open_dataset(os.path.join(ml_transform_path, valid_y_filename))

# 4 Processing

## 4.1 Stacking dimensions

In [38]:
train_x_xr_stack = train_x_xr.stack(sample=("realization","time"))
valid_x_xr_stack = valid_x_xr.stack(sample=("realization","time"))

In [39]:
train_y_xr_stack = train_y_xr.stack(sample=("realization","time"))
valid_y_xr_stack = valid_y_xr.stack(sample=("realization","time"))


In [40]:
valid_sample_coords = valid_y_xr_stack.sample
train_sample_coords = train_y_xr_stack.sample

## 4.2 Transform to numpy and mask nans with zeros

In [41]:
train_x_ml_np = np.nan_to_num(np.expand_dims(train_x_xr_stack["rho"].transpose("sample",...).values,3),0)
valid_x_ml_np = np.nan_to_num(np.expand_dims(valid_x_xr_stack["rho"].transpose("sample",...).values,3),0)

train_y_ml_np = train_y_xr_stack["atlantic_moc"].values
valid_y_ml_np = valid_y_xr_stack["atlantic_moc"].values

In [42]:
lon = train_x_xr_stack.lon
lat = train_x_xr_stack.lat

# 5 Model

In [43]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(30,(3,3), activation="relu", padding="same",input_shape=(120,121,1)))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add(tf.keras.layers.Conv2D(30,(3,3), activation="relu", padding="same", input_shape=(60,60,8)))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add(tf.keras.layers.Conv2D(30,(3,3), activation="relu", padding="same", input_shape=(30,30,32)))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Flatten())

#model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(50, activation="relu"))
#model.add(tf.keras.layers.BatchNormalization())

#model.add(tf.keras.layers.Dropout(0.2))
#model.add(tf.keras.layers.Dense(50, activation="relu"))
#model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(1, activation="linear"))

In [23]:
model.build()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 120, 121, 30)      300       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 60, 60, 30)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 60, 30)        8130      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 30)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 30, 30, 30)        8130      
                                                                 
 flatten (Flatten)           (None, 27000)             0

In [25]:
model_file_template = os.path.join(model_path, "saved-model-{epoch:02d}.hdf5")
checkpoint = tf.keras.callbacks.ModelCheckpoint(model_file_template, monitor='val_mse', verbose=1, save_best_only=False, mode='max')

In [26]:
from tensorflow.keras.callbacks import Callback

class TrainValMSEDiff(Callback):
    
    def on_epoch_end(self, epoch, logs={}):
        train_mse = model.evaluate(train_x_ml_np, train_y_ml_np)
        valid_mse = model.evaluate(valid_x_ml_np, valid_y_ml_np)
        diff_mse  = train_mse - valid_mse
        
        logs["train_mse_epoch_end"] = train_mse
        logs["valid_mse_epoch_end"] = valid_mse
        logs["diff_mse_epoch_end"] = diff_mse
        print(f'Epoch {epoch+1} - train_mse: {train_mse:.4f} - val_mse: {valid_mse:.4f} - diff: {diff_mse:.4f}')


In [27]:
csv_logger = tf.keras.callbacks.CSVLogger(os.path.join(model_path, "history.csv"))

In [28]:
logdir = os.path.join("logs", model_name)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = os.path.join(model_path,logdir))

# Train Model

In [29]:
opt = tf.keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(optimizer = opt, loss=tf.keras.losses.mse)

In [30]:
model.fit(x=train_x_ml_np, y=train_y_ml_np, batch_size=512, epochs=200, validation_data=(valid_x_ml_np, valid_y_ml_np), callbacks = [TrainValMSEDiff(), checkpoint, csv_logger, tensorboard_callback])


Train on 13600 samples, validate on 2000 samples
Epoch 1/200


2023-02-28 10:45:08.741869: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 10:45:08.896763: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-02-28 10:45:08.951552: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


  512/13600 [>.............................] - ETA: 1:07 - loss: 391.3251

2023-02-28 10:45:11.404922: I tensorflow/core/profiler/lib/profiler_session.cc:101] Profiler session initializing.
2023-02-28 10:45:11.404974: I tensorflow/core/profiler/lib/profiler_session.cc:116] Profiler session started.


 1024/13600 [=>............................] - ETA: 43s - loss: 387.1126 

2023-02-28 10:45:12.086004: I tensorflow/core/profiler/lib/profiler_session.cc:67] Profiler session collecting data.
2023-02-28 10:45:12.087735: I tensorflow/core/profiler/lib/profiler_session.cc:128] Profiler session tear down.
2023-02-28 10:45:12.202815: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: /work/uo1075/u301101/Doktorarbeit/CNN/models/cv_samplestandardized/28_02_2023_10_42_25/conv_level_23_kfold_0_date_28_02_2023_10_42_25/logs/conv_level_23_kfold_0_date_28_02_2023_10_42_25/plugins/profile/2023_02_28_10_45_12

2023-02-28 10:45:12.205195: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to /work/uo1075/u301101/Doktorarbeit/CNN/models/cv_samplestandardized/28_02_2023_10_42_25/conv_level_23_kfold_0_date_28_02_2023_10_42_25/logs/conv_level_23_kfold_0_date_28_02_2023_10_42_25/plugins/profile/2023_02_28_10_45_12/l20407.lvt.dkrz.de.trace.json.gz
2023-02-28 10:45:12.217762: I tensorflow/core/profile

13600/13600 [==============================] - ETA: 0s - loss: 316.2692

/work/uo1075/u301101/programming/miniconda3/envs/CNN/lib/python3.8/site-packages/keras/engine/training_v1.py:2332: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


Epoch 1 - train_mse: 171.0116 - val_mse: 4.2940 - diff: 166.7176

Epoch 1: saving model to /work/uo1075/u301101/Doktorarbeit/CNN/models/cv_samplestandardized/28_02_2023_10_42_25/conv_level_23_kfold_0_date_28_02_2023_10_42_25/saved-model-01.hdf5
13600/13600 [==============================] - 73s 5ms/sample - loss: 316.2692 - val_loss: 4.2940 - train_mse_epoch_end: 171.0116 - valid_mse_epoch_end: 4.2940 - diff_mse_epoch_end: 166.7176
Epoch 2/200
13600/13600 [==============================] - ETA: 0s - loss: 46.8048Epoch 2 - train_mse: 8.0921 - val_mse: 1603.3290 - diff: -1595.2369

Epoch 2: saving model to /work/uo1075/u301101/Doktorarbeit/CNN/models/cv_samplestandardized/28_02_2023_10_42_25/conv_level_23_kfold_0_date_28_02_2023_10_42_25/saved-model-02.hdf5
13600/13600 [==============================] - 59s 4ms/sample - loss: 46.8048 - val_loss: 1603.3290 - train_mse_epoch_end: 8.0921 - valid_mse_epoch_end: 1603.3290 - diff_mse_epoch_end: -1595.2369
Epoch 3/200
13600/13600 [=============

KeyboardInterrupt: 